In [37]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from itertools import cycle

In [38]:
year_month = '202412'

In [39]:
# 解析年月字符串
year_month_dt = datetime.strptime(year_month, '%Y%m')

# 计算去年同期的年和月
previous_year = year_month_dt.year - 1
previous_month = year_month_dt.month

Mon = previous_month

# 格式化去年同期的年月
previous_year_month = f"{previous_year:04d}{previous_month:02d}"

# 计算去年同期月份的第一天
previous_month_first_day = year_month_dt.replace(year=previous_year, month=previous_month, day=1)

# 计算去年同期月份的最后一天
etd = (previous_month_first_day + timedelta(days=31)).replace(day=1) - timedelta(days=1)
etd = etd.strftime('%Y-%m-%d')

# 输出结果
print(f"Previous year month: {previous_year_month}")
print(f"ETD (End of the month of the previous year): {etd}")
print(Mon)

Previous year month: 202305
ETD (End of the month of the previous year): 2023-05-31
5


In [40]:
vendor_mapping = pd.read_excel(r'C:\Medline\2. CPM\data\vendor_mapping\Vendor _mapping 2024_v2.xlsx')
vendor_mapping['Vendor Number'] = vendor_mapping['Vendor Number'].map(str)
vendor_mapping_dict = dict(zip(vendor_mapping['Vendor Number'],vendor_mapping['Cleaned Vendor Name']))

In [41]:
vendor_mapping_dict = dict(zip(vendor_mapping['Vendor Number'],vendor_mapping['Cleaned Vendor Name']))
vendor_nation_dict = dict(zip(vendor_mapping['Cleaned Vendor Name'],vendor_mapping['Nation']))
vendor_manager_dict = dict(zip(vendor_mapping['Cleaned Vendor Name'],vendor_mapping['Regional Manager']))

In [42]:
blitz_vendor_info = pd.read_excel(r'C:\Medline\Vendor info\2024_BlitzVendor.xlsx',sheet_name=1)
blitz_ProjectLeader_dict = dict(zip(blitz_vendor_info['Vendor Name'],blitz_vendor_info['Project Leader']))
blitz_Category_dict = dict(zip(blitz_vendor_info['Vendor Name'],blitz_vendor_info['Category']))

In [43]:
# df_complaint_21_22 =pd.read_excel(r'C:\Medline\2. CPM\2. US Complaints\US Complaint Data 201901-202401.xlsx',sheet_name=1)
# df_complaint_21_22 = df_complaint_21_22.loc[df_complaint_21_22['Year'].isin([2021,2022])]

In [44]:
df_complaints_2024 = pd.read_excel(r'C:\Medline\2. CPM\2. US Complaints\2024\{} Complaint Data.xlsx'.format(year_month),sheet_name='2024 Complaint Database')

In [45]:
df_complaints_2023 = pd.read_excel(r'C:\Medline\2. CPM\2. US Complaints\unidentified\2023\{} Complaint Data.xlsx'.format(previous_year_month),sheet_name='2023 Complaint Database')

In [46]:
df_complaints = pd.concat([df_complaints_2024,df_complaints_2023],ignore_index=True)
df_complaints.fillna('Unidentified Vendor',inplace=True)

In [47]:
# df_blitz = pd.concat([df_complaints_2024,df_complaints_2023],ignore_index=True)
# df_blitz.to_excel('blitz_complaints.xlsx',index=False)

In [48]:
df_complaints_gy = (
    df_complaints
    .query("`If Manufacturing Complaint` == 'Y' and Year.isin([2021,2022,2023,2024])")
    .assign(Quarter =lambda d: d['Notification Created Date'].dt.quarter,
            Division = lambda d: d['Division'].map(int).map(str))
        .groupby(['VendorName','Division','Year','Quarter','Month'],as_index=False).size()
)
df_complaints_gy.rename(columns={'size':'Complaints'},inplace=True)

In [49]:
df_inspection_his = pd.read_excel(r'C:\Medline\3. QC trend report\Asia QC Trend Report 202001_202405.xlsx',sheet_name=0)

In [50]:
# df_inspection_us_2021_2022 =  df_inspection_his.loc[(df_inspection_his['Branch'] == 'US' ) & (df_inspection_his['Year'].isin([2021,2022])),['Vendors','Division','Year','Quarter','Month','#: Items inspected','#: Items reworked']]
df_inspection_us_2023 =  df_inspection_his.loc[(df_inspection_his['Branch'] == 'US' ) & (df_inspection_his['Year'].isin([2023])) & (df_inspection_his['Month']<= Mon),['Vendors','Division','Year','Quarter','Month','#: Items inspected','#: Items reworked']]

In [51]:
df_inspection_us_current = pd.read_excel(r'C:\Medline\3. QC trend report\2024\Excel Report\Asia QC Trend Report US_202405.xlsx',sheet_name='Overall')
df_inspection_us_current =  df_inspection_us_current.loc[(df_inspection_us_current['Branch'] == 'US' ) & (df_inspection_us_current['Year'] == 2024),['Vendors','Division','Year','Quarter','Month','#: Items inspected','#: Items reworked']]

In [52]:
df_inspection = pd.concat([df_inspection_us_current,df_inspection_us_2023],ignore_index=True)
df_inspection_us_groupby = df_inspection.groupby(['Vendors','Division','Year','Quarter','Month'],as_index=False).sum()

In [53]:
df_inspection_us_groupby.rename(columns={'Vendors':'VendorName'},inplace=True)

In [54]:
df_inspection_us_groupby['Year'] = df_inspection_us_groupby['Year'].map(int)
df_inspection_us_groupby['Quarter'] = df_inspection_us_groupby['Quarter'].map(int)
df_inspection_us_groupby['Month'] = df_inspection_us_groupby['Month'].map(int)
df_inspection_us_groupby['Division'] = df_inspection_us_groupby['Division'].map(int).map(str)

In [55]:
# df_BUOM_2021  = pd.read_excel(r'C:\Medline\2. CPM\4. CPM by BUOM\BUOM Database\history\US_ShippingData_2021.xlsx', usecols='D,E,AB,AP,BI')
# df_BUOM_2022  = pd.read_excel(r'C:\Medline\2. CPM\4. CPM by BUOM\BUOM Database\history\US_ShippingData_2022.xlsx', usecols='D,E,AB,AP,BI')

In [56]:
df_BUOM_2023  = pd.read_excel(r'C:\Medline\2. CPM\4. CPM by BUOM\BUOM Database\2023\US BUOM {}.xlsx'.format(previous_year_month), usecols='C,D,H,P,Q')
df_BUOM_2023 = df_BUOM_2023.loc[(df_BUOM_2023['ShipmentDt'] <= etd) & (df_BUOM_2023['ShipmentDt'] >= '2023-01-01')]

In [57]:
df_BUOM_2024  = pd.read_excel(r'C:\Medline\2. CPM\4. CPM by BUOM\BUOM Database\2024\US\US BUOM {}.xlsx'.format(year_month), usecols='D,E,P,X,Y')

In [58]:
df_us_buom = pd.concat([df_BUOM_2024,df_BUOM_2023],ignore_index=True)

In [59]:
df_us_buom.dropna(subset=['StrippedVendorNumber'],inplace=True)
df_us_buom['Year'] = df_us_buom['ShipmentDt'].dt.year
df_us_buom['Quarter'] = df_us_buom['ShipmentDt'].dt.quarter
df_us_buom['Month'] = df_us_buom['ShipmentDt'].dt.month

In [60]:
df_us_buom.rename(columns={'StrippedVendorNumber':'VendorNumber'},inplace=True)
df_us_buom['VendorNumber'] = df_us_buom['VendorNumber'].map(int).map(str)

In [61]:
df_us_buom_all = df_us_buom.copy()

In [62]:
vendor_mapping_inspection = vendor_mapping.loc[vendor_mapping['Is_Asia']=='Y','Vendor Number'].to_list()
df_us_buom_all = df_us_buom_all.loc[df_us_buom_all['VendorNumber'].isin(vendor_mapping_inspection)]
df_us_buom_all['VendorName'] = df_us_buom_all['VendorNumber'].apply(lambda x : vendor_mapping_dict.get(x,np.nan))
df_us_buom_all.rename(columns={'ProductDivisionCode':'Division'},inplace=True)
df_us_buom_all['Division'] = df_us_buom_all['Division'].map(int).map(str)

In [63]:
df_us_buom_gy = df_us_buom_all.groupby(['VendorName','Division','Year','Quarter','Month'],as_index=False)['BUOMShipmentQty'].sum()

In [64]:
df_us_buom_gy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6575 entries, 0 to 6574
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   VendorName       6575 non-null   object
 1   Division         6575 non-null   object
 2   Year             6575 non-null   int64 
 3   Quarter          6575 non-null   int64 
 4   Month            6575 non-null   int64 
 5   BUOMShipmentQty  6575 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 308.3+ KB


In [65]:
df_complaints_gy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1018 entries, 0 to 1017
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   VendorName  1018 non-null   object
 1   Division    1018 non-null   object
 2   Year        1018 non-null   int64 
 3   Quarter     1018 non-null   int64 
 4   Month       1018 non-null   int64 
 5   Complaints  1018 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.8+ KB


In [66]:
df_inspection_us_groupby.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6914 entries, 0 to 6913
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   VendorName          6914 non-null   object
 1   Division            6914 non-null   object
 2   Year                6914 non-null   int64 
 3   Quarter             6914 non-null   int64 
 4   Month               6914 non-null   int64 
 5   #: Items inspected  6914 non-null   int64 
 6   #: Items reworked   6914 non-null   int64 
dtypes: int64(5), object(2)
memory usage: 378.2+ KB


In [67]:
raw_data = df_complaints_gy.merge(df_us_buom_gy,how ='outer',on=['VendorName','Division','Year','Quarter','Month']).merge(df_inspection_us_groupby,how = 'outer',on=['VendorName','Division','Year','Quarter','Month']).fillna(0)

In [68]:
raw_data['Nation'] = raw_data['VendorName'].apply(lambda x : vendor_nation_dict.get(x,np.nan))
raw_data['Regional Manager'] = raw_data['VendorName'].apply(lambda x : vendor_manager_dict.get(x,np.nan))
raw_data['Project Leader'] = raw_data['VendorName'].apply(lambda x : blitz_ProjectLeader_dict.get(x,np.nan))
raw_data['Category'] = raw_data['VendorName'].apply(lambda x : blitz_Category_dict.get(x,np.nan))
raw_data['CPM'] = None

In [69]:
raw_data.head()

,VendorName,Division,Year,Quarter,Month,Complaints,BUOMShipmentQty,#: Items inspected,#: Items reworked,Nation,Regional Manager,Project Leader,Category,CPM
0,A AND J HOLDING CO LTD,30,2023,1,1,4.0,0.0,0.0,0.0,CN,Richard Li(DME),Gaming Chen,Top 12 Vendors,None
1,A AND J HOLDING CO LTD,32,2023,1,1,1.0,794.0,3.0,0.0,CN,Richard Li(DME),Gaming Chen,Top 12 Vendors,None
2,A AND J HOLDING CO LTD,32,2023,1,2,1.0,480.0,8.0,0.0,CN,Richard Li(DME),Gaming Chen,Top 12 Vendors,None
3,A AND J HOLDING CO LTD,32,2023,1,3,1.0,1173.0,5.0,0.0,CN,Richard Li(DME),Gaming Chen,Top 12 Vendors,None
4,A AND J HOLDING CO LTD,32,2024,1,1,1.0,1057.0,5.0,0.0,CN,Richard Li(DME),Gaming Chen,Top 12 Vendors,None


In [70]:
raw_data = raw_data[['VendorName',
                     'Nation',
                     'Project Leader',
                     'Regional Manager',
                     'Category',
                     'Year',
                     'Month',
                     'Quarter',
                     'Division',
                     'Complaints',
                     'BUOMShipmentQty',
                     'CPM',
                     '#: Items inspected',
                     '#: Items reworked'
                     ]]

In [71]:
raw_data = raw_data[~(raw_data['Complaints'] + raw_data['BUOMShipmentQty'] + raw_data['#: Items inspected'] + raw_data['#: Items reworked'] == 0)]

In [72]:
raw_data.to_excel(f'us_raw_data_{year_month}.xlsx')